# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f91304a1af0>
├── 'a' --> tensor([[-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255]])
└── 'x' --> <FastTreeValue 0x7f91304a1400>
    └── 'c' --> tensor([[ 0.4786, -0.7946,  0.4894,  0.2897],
                        [-0.8938, -0.8693,  0.6544,  0.5854],
                        [ 0.7453,  0.0114,  0.5084,  0.7695]])

In [4]:
t.a

tensor([[-1.0451, -1.3092,  0.2996],
        [-0.5193,  0.3001, -0.3255]])

In [5]:
%timeit t.a

75.8 ns ± 0.0465 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f91304a1af0>
├── 'a' --> tensor([[-0.3947, -0.7862,  0.7363],
│                   [ 0.6211, -0.8427,  0.0144]])
└── 'x' --> <FastTreeValue 0x7f91304a1400>
    └── 'c' --> tensor([[ 0.4786, -0.7946,  0.4894,  0.2897],
                        [-0.8938, -0.8693,  0.6544,  0.5854],
                        [ 0.7453,  0.0114,  0.5084,  0.7695]])

In [7]:
%timeit t.a = new_value

87.8 ns ± 0.0855 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255]]),
    x: Batch(
           c: tensor([[ 0.4786, -0.7946,  0.4894,  0.2897],
                      [-0.8938, -0.8693,  0.6544,  0.5854],
                      [ 0.7453,  0.0114,  0.5084,  0.7695]]),
       ),
)

In [10]:
b.a

tensor([[-1.0451, -1.3092,  0.2996],
        [-0.5193,  0.3001, -0.3255]])

In [11]:
%timeit b.a

66.1 ns ± 0.108 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0279,  0.5048, -0.8348],
               [-0.9516,  0.8101,  0.8461]]),
    x: Batch(
           c: tensor([[ 0.4786, -0.7946,  0.4894,  0.2897],
                      [-0.8938, -0.8693,  0.6544,  0.5854],
                      [ 0.7453,  0.0114,  0.5084,  0.7695]]),
       ),
)

In [13]:
%timeit b.a = new_value

545 ns ± 0.0921 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

883 ns ± 6.59 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 36.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

150 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

151 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f905a24ddc0>
├── 'a' --> tensor([[[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]],
│           
│                   [[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]],
│           
│                   [[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]],
│           
│                   [[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]],
│           
│                   [[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]],
│           
│                   [[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]],
│           
│                   [[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]],
│           
│                   [[-1.0451, -1.3092,  0.2996],
│                    [-0.5193,  0.3001, -0.3255]]])
└── 'x' --> <FastTreeValue 0x7f904f31e520>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

34.1 µs ± 41.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9046bd4c40>
├── 'a' --> tensor([[-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255],
│                   [-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255],
│                   [-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255],
│                   [-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255],
│                   [-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255],
│                   [-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255],
│                   [-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255],
│                   [-1.0451, -1.3092,  0.2996],
│                   [-0.5193,  0.3001, -0.3255]])
└── 'x' --> <FastTreeValue 0x7f9046bd4e50>
    └── 'c' --> tensor([[ 0.4786, -0.7946,  0.4894,  0.2897],
                        [-0.8938, -0.8693,  0.6544,  0.5854],
                 

In [23]:
%timeit t_cat(trees)

32.4 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.7 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]],
       
               [[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]],
       
               [[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]],
       
               [[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]],
       
               [[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]],
       
               [[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]],
       
               [[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]],
       
               [[-1.0451, -1.3092,  0.2996],
                [-0.5193,  0.3001, -0.3255]]]),
    x: Batch(
           c: tensor([[[ 0.4786, -0.7946,  0.4894,  0.2897],
                       [-0.8938, -0.8693,  0.6544,  0.5854],
                       [ 0.7453,  0.0114,  0.5084,  0.7695]],
         

In [26]:
%timeit Batch.stack(batches)

84.3 µs ± 176 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255],
               [-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255],
               [-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255],
               [-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255],
               [-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255],
               [-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255],
               [-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255],
               [-1.0451, -1.3092,  0.2996],
               [-0.5193,  0.3001, -0.3255]]),
    x: Batch(
           c: tensor([[ 0.4786, -0.7946,  0.4894,  0.2897],
                      [-0.8938, -0.8693,  0.6544,  0.5854],
                      [ 0.7453,  0.0114,  0.5084,  0.7695],
                      [ 0.4786, -0.7946,  0.4894,  0.2897],
                      [-0.8938, -

In [28]:
%timeit Batch.cat(batches)

154 µs ± 247 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

376 µs ± 5.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
